In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
class SentimentDataset(Dataset):
    def __init__(self, vectors, labels):
        self.vectors = torch.FloatTensor(vectors)
        self.labels = torch.FloatTensor(labels)

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.vectors[idx], self.labels[idx]


In [3]:
class LSTMSentiment(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.5):
        super(LSTMSentiment, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, (h_n, _) = self.lstm(x)
        lstm_out = self.dropout(lstm_out)
        out = self.fc(h_n[-1])  # Use the last layer's hidden state
        out = self.sigmoid(out)
        return out

In [4]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.001):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 5
            min_delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                               Default: 0
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [5]:
# Train Model Function
def train_model(model, train_loader, val_loader, criterion, optimizer, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for vectors, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            vectors, labels = vectors.to("cpu"), labels.to("cpu").float()
            
            optimizer.zero_grad()
            output = model(vectors.unsqueeze(1))  # Adding sequence dimension
            loss = criterion(output.squeeze(), labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for vectors, labels in val_loader:
                vectors, labels = vectors.to("cpu"), labels.to("cpu")
                output = model(vectors.unsqueeze(1))  # Adding sequence dimension
                loss = criterion(output.squeeze(), labels)
                val_loss += loss.item()
                predicted = (output.squeeze() > 0.5).float()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        print(f"Epoch {epoch+1}/{n_epochs}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}")
        print(f"Val Accuracy: {correct/total:.4f}")
        print()

In [6]:
vectors = pd.read_csv(r'C:\Learning\Machine-Learning\Deep_Learning_WorkSpace\projects\sentiment-analysis-project\data\processed\vector10000.csv')
labels = pd.read_csv(r'C:\Learning\Machine-Learning\Deep_Learning_WorkSpace\projects\sentiment-analysis-project\data\raw\label10000.csv')
labels = labels['label']

In [7]:
vectors.shape, labels.shape

((10000, 300), (10000,))

In [8]:
print(labels.min(), labels.max())
print(labels.unique())

0 1
[0 1]


In [9]:
labels.sample(15)

1455    0
4793    0
5335    1
74      0
7227    1
3650    0
142     0
128     0
9980    1
141     0
7609    1
4571    0
4140    0
5279    1
1523    0
Name: label, dtype: int64

In [10]:
X_train, X_val, y_train, y_val = train_test_split(vectors, labels, test_size=0.2, random_state=42)

In [11]:
type(X_train)

pandas.core.frame.DataFrame

In [12]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_val = np.array(X_val)
y_val = np.array(y_val)

In [13]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((8000, 300), (2000, 300), (8000,), (2000,))

In [14]:
train_dataset = SentimentDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = SentimentDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [15]:
# Initialize model
input_dim = 300  # Word2Vec dimension
hidden_dim = 128
output_dim = 1  # Binary classification
n_layers = 2

In [16]:
model = LSTMSentiment(input_dim, hidden_dim, output_dim, n_layers)
model.to('cpu')

LSTMSentiment(
  (lstm): LSTM(300, 128, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [17]:
# Define loss function and optimizer
criterion = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, n_epochs=3)

Epoch 1/3: 100%|██████████| 250/250 [00:02<00:00, 105.72it/s]


Epoch 1/3
Train Loss: 0.6122
Val Loss: 0.5697
Val Accuracy: 0.7080



Epoch 2/3: 100%|██████████| 250/250 [00:02<00:00, 107.42it/s]


Epoch 2/3
Train Loss: 0.5521
Val Loss: 0.5618
Val Accuracy: 0.7190



Epoch 3/3: 100%|██████████| 250/250 [00:02<00:00, 108.04it/s]


Epoch 3/3
Train Loss: 0.5405
Val Loss: 0.5686
Val Accuracy: 0.6990



In [20]:
 # Save the model
torch.save(model.state_dict(), "C:\Learning\Machine-Learning\Deep_Learning_WorkSpace\projects\sentiment-analysis-project\\backend\models\lstm_sentiment_model.pth")
print("Model saved successfully.")

Model saved successfully.


In [106]:
def predict_sentiment(model, vector):
    model.eval()
    
    # Disable gradient calculations (not needed for inference)
    with torch.no_grad():
        # Add batch dimension to the input vector (shape: [1, 1, sequence_length, input_size])
        vector = vector.unsqueeze(0)
        
        # Pass the vector through the model
        output = model(vector)
        
        # Apply softmax to get the probabilities and then take the argmax to get the predicted class
        if output > 0.5 :
            return 1
        elif output <0.5 :
            return 0
        else:
            return "unexpected output"        
